In [ ]:
!pip -q install "mlflow>=3.0.0" scikit-learn pandas numpy matplotlib requests

import os
import time
import json
import shutil
import socket
import signal
import subprocess
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
)

import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

def _is_port_open(host: str, port: int, timeout_s: float = 0.2) -> bool:
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.settimeout(timeout_s)
        return s.connect_ex((host, port)) == 0

def _wait_for_http(url: str, timeout_s: int = 30) -> None:
    t0 = time.time()
    last_err = None
    while time.time() - t0 < timeout_s:
        try:
            r = requests.get(url, timeout=1)
            if r.status_code < 500:
                return
        except Exception as e:
            last_err = e
        time.sleep(0.5)
    raise RuntimeError(f"Server not ready at {url}. Last error: {last_err}")

def _safe_kill(proc: subprocess.Popen):
    if proc is None:
        return
    try:
        proc.terminate()
        try:
            proc.wait(timeout=5)
        except subprocess.TimeoutExpired:
            proc.kill()
    except Exception:
        pass

In [ ]:
BASE_DIR = Path("/content/mlflow_colab_demo").resolve()
BACKEND_DB = BASE_DIR / "mlflow.db"
ARTIFACT_ROOT = BASE_DIR / "mlartifacts"
os.makedirs(BASE_DIR, exist_ok=True)
os.makedirs(ARTIFACT_ROOT, exist_ok=True)

HOST = "127.0.0.1"
PORT = 5000
TRACKING_URI = f"http://{HOST}:{PORT}"

if _is_port_open(HOST, PORT):
    for p in range(5001, 5015):
        if not _is_port_open(HOST, p):
            PORT = p
            TRACKING_URI = f"http://{HOST}:{PORT}"
            break

print("Using TRACKING_URI:", TRACKING_URI)
print("Backend DB:", BACKEND_DB)
print("Artifact root:", ARTIFACT_ROOT)

server_cmd = [
    "mlflow",
    "server",
    "--host", HOST,
    "--port", str(PORT),
    "--backend-store-uri", f"sqlite:///{BACKEND_DB}",
    "--default-artifact-root", str(ARTIFACT_ROOT),
]

mlflow_server = subprocess.Popen(
    server_cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)

_wait_for_http(TRACKING_URI, timeout_s=45)
mlflow.set_tracking_uri(TRACKING_URI)
print("MLflow server is up.")

EXPERIMENT_NAME = "colab-advanced-mlflow-sklearn"
mlflow.set_experiment(EXPERIMENT_NAME)

In [ ]:
data = load_breast_cancer(as_frame=True)
df = data.frame.copy()
target_col = "target"

X = df.drop(columns=[target_col])
y = df[target_col].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

mlflow.sklearn.autolog(
    log_input_examples=False,
    log_model_signatures=False,
    silent=True
)

C_VALUES = [0.01, 0.1, 1.0, 3.0]
SOLVERS = ["liblinear", "lbfgs"]

best = {"auc": -1.0, "run_id": None, "params": None}

In [ ]:
with mlflow.start_run(run_name="parent_sweep_run") as parent_run:
    mlflow.log_param("dataset", "sklearn_breast_cancer")
    mlflow.log_param("n_features", X_train.shape[1])
    mlflow.log_param("n_train", X_train.shape[0])
    mlflow.log_param("n_test", X_test.shape[0])

    for C in C_VALUES:
        for solver in SOLVERS:
            with mlflow.start_run(run_name=f"child_C={C}_solver={solver}", nested=True) as child_run:
                pipe = Pipeline([
                    ("scaler", StandardScaler()),
                    ("clf", LogisticRegression(
                        C=C,
                        solver=solver,
                        penalty="l2",
                        max_iter=2000,
                        random_state=42
                    ))
                ])

                pipe.fit(X_train, y_train)
                proba = pipe.predict_proba(X_test)[:, 1]
                pred = (proba >= 0.5).astype(int)

                auc = roc_auc_score(y_test, proba)
                acc = accuracy_score(y_test, pred)
                prec = precision_score(y_test, pred, zero_division=0)
                rec = recall_score(y_test, pred, zero_division=0)
                f1 = f1_score(y_test, pred, zero_division=0)

                mlflow.log_metrics({
                    "test_auc": float(auc),
                    "test_accuracy": float(acc),
                    "test_precision": float(prec),
                    "test_recall": float(rec),
                    "test_f1": float(f1),
                })

                cm = confusion_matrix(y_test, pred)
                disp = ConfusionMatrixDisplay(cm, display_labels=data.target_names)
                fig, ax = plt.subplots(figsize=(5, 4))
                disp.plot(ax=ax, values_format="d")
                ax.set_title(f"Confusion Matrix (C={C}, solver={solver})")
                cm_path = BASE_DIR / "confusion_matrix.png"
                fig.tight_layout()
                fig.savefig(cm_path, dpi=140)
                plt.close(fig)
                mlflow.log_artifact(str(cm_path), artifact_path="diagnostics")

                if auc > best["auc"]:
                    best.update({
                        "auc": float(auc),
                        "run_id": child_run.info.run_id,
                        "params": {"C": C, "solver": solver}
                    })

    mlflow.log_dict(best, "best_run_summary.json")
    print("Best config:", best)

In [ ]:
best_C = best["params"]["C"]
best_solver = best["params"]["solver"]

final_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(
        C=best_C,
        solver=best_solver,
        penalty="l2",
        max_iter=2000,
        random_state=42
    ))
])

with mlflow.start_run(run_name="final_model_run") as final_run:
    final_pipe.fit(X_train, y_train)

    proba = final_pipe.predict_proba(X_test)[:, 1]
    pred = (proba >= 0.5).astype(int)

    metrics = {
        "test_auc": float(roc_auc_score(y_test, proba)),
        "test_accuracy": float(accuracy_score(y_test, pred)),
        "test_precision": float(precision_score(y_test, pred, zero_division=0)),
        "test_recall": float(recall_score(y_test, pred, zero_division=0)),
        "test_f1": float(f1_score(y_test, pred, zero_division=0)),
    }
    mlflow.log_metrics(metrics)
    mlflow.log_params({"C": best_C, "solver": best_solver, "model": "LogisticRegression+StandardScaler"})

    input_example = X_test.iloc[:5].copy()
    signature = infer_signature(input_example, final_pipe.predict_proba(input_example)[:, 1])

    model_info = mlflow.sklearn.log_model(
        sk_model=final_pipe,
        artifact_path="model",
        signature=signature,
        input_example=input_example,
        registered_model_name=None,
    )

    print("Final run_id:", final_run.info.run_id)
    print("Logged model URI:", model_info.model_uri)

    eval_df = X_test.copy()
    eval_df["label"] = y_test.values

    eval_result = mlflow.models.evaluate(
        model=model_info.model_uri,
        data=eval_df,
        targets="label",
        model_type="classifier",
        evaluators="default",
    )

    eval_summary = {
        "metrics": {k: float(v) if isinstance(v, (int, float, np.floating)) else str(v)
                    for k, v in eval_result.metrics.items()},
        "artifacts": {k: str(v) for k, v in eval_result.artifacts.items()},
    }
    mlflow.log_dict(eval_summary, "evaluation/eval_summary.json")

In [1]:
SERVE_PORT = 6000
if _is_port_open(HOST, SERVE_PORT):
    for p in range(6001, 6020):
        if not _is_port_open(HOST, p):
            SERVE_PORT = p
            break

MODEL_URI = model_info.model_uri

serve_cmd = [
    "mlflow", "models", "serve",
    "-m", MODEL_URI,
    "-p", str(SERVE_PORT),
    "--host", HOST,
    "--env-manager", "local"
]

mlflow_serve = subprocess.Popen(
    serve_cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)

serve_url = f"http://{HOST}:{SERVE_PORT}/invocations"
_wait_for_http(f"http://{HOST}:{SERVE_PORT}", timeout_s=60)
print("Model server is up at:", serve_url)

payload = {
    "dataframe_split": {
        "columns": list(X_test.columns),
        "data": X_test.iloc[:3].values.tolist()
    }
}

r = requests.post(
    serve_url,
    headers={"Content-Type": "application/json"},
    data=json.dumps(payload),
    timeout=10
)
print("Serve status:", r.status_code)
print("Predictions (probabilities or outputs):", r.text)

print("\nOpen the MLflow UI by visiting:", TRACKING_URI)
print("Artifacts are stored under:", ARTIFACT_ROOT)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.9 MB/s eta 0:00:00
Using TRACKING_URI: http://127.0.0.1:5000
Backend DB

2026/02/06 18:51:43 INFO mlflow.tracking.fluent: Experiment with name 'colab-advanced-mlflow-sklearn' does not exist. Creating a new experiment.


🏃 View run child_C=0.01_solver=liblinear at: http://127.0.0.1:5000/#/experiments/1/runs/5369f6209af84e018198717442c6118a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
🏃 View run child_C=0.01_solver=lbfgs at: http://127.0.0.1:5000/#/experiments/1/runs/106619f400eb45568a990569fab72ff0
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
🏃 View run child_C=0.1_solver=liblinear at: http://127.0.0.1:5000/#/experiments/1/runs/f48f1af6bf1642098bf31d97bd6cc4db
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
🏃 View run child_C=0.1_solver=lbfgs at: http://127.0.0.1:5000/#/experiments/1/runs/ad1725fcd44b4aacac88f81af05124ad
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
🏃 View run child_C=1.0_solver=liblinear at: http://127.0.0.1:5000/#/experiments/1/runs/34f337f003dd45109376b6e58ffa2dff
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
🏃 View run child_C=1.0_solver=lbfgs at: http://127.0.0.1:5000/#/experiments/1/runs/2fd741a1bd444a37b511460

2026/02/06 18:52:40 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
2026/02/06 18:52:44 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-0e957a3fb4e847ecad4e7a5d31f8b60a
2026/02/06 18:52:44 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.


Final run_id: 212e682de11045dcb4cc0916d21a810d
Logged model URI: models:/m-0e957a3fb4e847ecad4e7a5d31f8b60a


/usr/local/lib/python3.12/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2026/02/06 18:52:44 WARNING mlflow.models.evaluation.evaluators.classifier: According to the evaluation dataset label values, the model type looks like None, but you specified model type 'classifier'. Please verify that you set the `model_type` and

🏃 View run final_model_run at: http://127.0.0.1:5000/#/experiments/1/runs/212e682de11045dcb4cc0916d21a810d
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1
Model server is up at: http://127.0.0.1:6000/invocations
Serve status: 200
Predictions (probabilities or outputs): {"predictions": [0, 1, 0]}

Open the MLflow UI by visiting: http://127.0.0.1:5000
Artifacts are stored under: /content/mlflow_colab_demo/mlartifacts
